### 1. Survey Combiner
##### This script combines all ECW data and executes the methods outline in the flow charts perscribed in the data methods folder. Throughout the script exerpts are commented out but can be uncommeneted if you wish to see the data at that stage.

In [100]:
import pandas as pd
import country_converter as coco
import sys
import os
from pathlib import Path
cc = coco.CountryConverter()

# create file paths
cwd = Path.cwd()
data_folder = cwd.parent / "data"
isco_08_poll_binary_PATH = data_folder / "ISCO-08 OpinionPollCensus.xlsx"
onet_context_by_soc_code_PATH = data_folder / "Indoors_Environmentally_Controlled_data.csv"
soc_isco_crosswalk_PATH = data_folder / "ISCO_SOC_Crosswalk.csv"
ilo_employment_by_isco_08_l2_PATH = data_folder / "ILO_ISCO_08_GLB.csv"
world_bank_labour_force_by_country_PATH = data_folder / "LFData_WB_plus.xlsx"



#### 1.1Reading/Processing ISCO-08 Poll results from the complete excel (found in the data folder)

In [101]:
df = pd.read_excel(isco_08_poll_binary_PATH, engine="openpyxl")
isco_08_poll_binary_df = df[['ISCO-08','Census']]
isco_08_poll_binary_df['ISCO-08'] = isco_08_poll_binary_df['ISCO-08'].astype(str).str.zfill(4)
isco_08_poll_binary_df = isco_08_poll_binary_df.set_index('ISCO-08')

C:\Users\seabr\AppData\Local\Temp\ipykernel_3428\817295888.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isco_08_poll_binary_df['ISCO-08'] = isco_08_poll_binary_df['ISCO-08'].astype(str).str.zfill(4)


In [102]:
#isco_08_poll_binary_df

#### 1.2 Reading/Processing ONET data on Indoor/Outdoor Context (found in the Data folder)

In [103]:
onet_context_by_soc_code_df = pd.read_csv(onet_context_by_soc_code_PATH)
onet_context_by_soc_code_df = onet_context_by_soc_code_df[['Context','Code']] # only keep nessesary columns
# NO ARMY INCLUDED, Use Blue Print results (%40 of Amry considered Critical)

#### 1.3 Reading/Processing SOC - ISCO-08 crosswalk, i.e. the map between codes (found in the Data folder)

In [104]:
soc_isco_crosswalk_df = pd.read_csv(soc_isco_crosswalk_PATH)
soc_isco_crosswalk_df = soc_isco_crosswalk_df[['2010 SOC Code','ISCO-08 Code']]
soc_isco_crosswalk_df['ISCO-08 Code'] = soc_isco_crosswalk_df['ISCO-08 Code'].astype(str)
# '''Bureau of Labor Statistics,,,,,
# On behalf of the Standard Occupational Classification Policy Committee (SOCPC),,,,,
# ,,,,,
# August 2012 (Updated June 2015),,,,,
# Questions should be emailed to soc@bls.gov'''
# note some are mapped to level 3 ISCO-08 codes, hence it should be truncated

In [105]:
#soc_isco_crosswalk_df

In [106]:
soc_isco_crosswalk_dict = soc_isco_crosswalk_df.set_index('2010 SOC Code')['ISCO-08 Code'].to_dict()

In [107]:
onet_context_by_soc_code_df['Code'] = onet_context_by_soc_code_df['Code'].astype(str).str.split('.').str[0]
onet_context_by_soc_code_df['Context'] = onet_context_by_soc_code_df['Context']/100
onet_context_by_soc_code_dict = onet_context_by_soc_code_df.set_index('Code')['Context'].to_dict()

In [108]:
ISCO_CONTEXT_InOutdoor_dict = {}
ONET_CODES_NOT_IN_MAP_dict = {} # if the Onet codes is not in the SOC Map it will be attached to a list of codes in the same 2-digit SOC group and added to the ISCO-08 2 digit codes further on
for k , v in onet_context_by_soc_code_dict.items():
  try:
    if k in ISCO_CONTEXT_InOutdoor_dict.keys(): soc_isco_crosswalk_dict[k].append(v)
    else: ISCO_CONTEXT_InOutdoor_dict[soc_isco_crosswalk_dict[k]] = [v]
  except:
    if k[:2] not in ONET_CODES_NOT_IN_MAP_dict.keys(): ONET_CODES_NOT_IN_MAP_dict[k[:2]] = [v]
    else: ONET_CODES_NOT_IN_MAP_dict[k[:2]].append(v)
    for i,v in soc_isco_crosswalk_dict.items():
      zone_list = []
      if k[:2] in i:
        zone_list.append(v)

L2_SOC_L2_ISCO_map = {}
for k in soc_isco_crosswalk_dict.keys():
  if k[:2] in ONET_CODES_NOT_IN_MAP_dict.keys():
    if k[:2] not in L2_SOC_L2_ISCO_map.keys(): L2_SOC_L2_ISCO_map[k[:2]] = [soc_isco_crosswalk_dict[k][:2]]
    else: L2_SOC_L2_ISCO_map[k[:2]].append(soc_isco_crosswalk_dict[k][:2])

L2_ISCO_MISSING = {}
for k, v in ONET_CODES_NOT_IN_MAP_dict.items():
  L2_ISCO_MISSING[max(L2_SOC_L2_ISCO_map[k], key=L2_SOC_L2_ISCO_map[k].count)] = v

In [109]:
ISCO_CONTEXT_InOutdoor_df = pd.DataFrame.from_dict(ISCO_CONTEXT_InOutdoor_dict, orient='index', columns=['Context'])
ISCO_CONTEXT_InOutdoor_df.index.name = 'ISCO-08 Code'

#### 1.4 Collapse the Indoor / Outdoor Context Data to the Level 2 ISCO codes


In [110]:
abs_level = 2  # Truncated to the 2 digit level

# Ensure codes are of the same format before mereg
L = isco_08_poll_binary_df.copy()
R = ISCO_CONTEXT_InOutdoor_df.copy()
L.index = L.index.astype(str).str.zfill(4)
R.index = R.index.astype(str).str.zfill(4)

# Merge
df = pd.merge(L, R, left_index=True, right_index=True, how='left')

# Clean up null errors
df['Context'] = pd.to_numeric(df.get('Context'), errors='coerce')
df['Census']  = pd.to_numeric(df.get('Census'),  errors='coerce')

# Collapse by the mean of its level, if not take the mean of the level above
df['_L3'] = df.index.str[:3]
df['_L2'] = df.index.str[:2]
df['_L1'] = df.index.str[:1]

mean_L3  = df.groupby('_L3')['Context'].transform('mean')
mean_L2  = df.groupby('_L2')['Context'].transform('mean')
mean_L1  = df.groupby('_L1')['Context'].transform('mean')
mean_all = df['Context'].mean()

df['Context Proj'] = (
    df['Context']
      .fillna(mean_L3)
      .fillna(mean_L2)
      .fillna(mean_L1)
)


# Manual overrides
overrides = ['0110', '0210', '0310']
df.loc[overrides, 'Context Proj'] = 1

# Tidy Up
df.drop(columns=['_L3','_L2','_L1', 'Context'], inplace=True)

ISCO_LVL4_ECW_WEIGHTS = df

In [111]:
#ISCO_LVL4_ECW_WEIGHTS

In [112]:
abs_level = 2  # Truncte to the 2 digit level

df = ISCO_LVL4_ECW_WEIGHTS.copy()
df.index = df.index.astype(str).str.zfill(4)

# Group to level 2
ISCO_LVL2_ECW_WEIGHTS = (
    df.assign(ISCO_trunc=df.index.str[:abs_level])
      .groupby('ISCO_trunc')
      .agg({
          'Census': 'mean',
          'Context Proj': 'mean'
      })
      .rename_axis('ISCO-08')
      .rename(columns={'Census': 'Critical Weight POLL'})
)

# Add Critical Weights from the ILO paper for the upper bound.
# Binary 1 / 0 rather then continuous values
ILO_LVL2_ECW_GROUPS = [61,62,63,92,94,22,32,53,52,95,54,71,72,73,74,75,81,82,93,91,96,83,31,44,51,1,2,3]

# Ensure entrys are integers
ISCO_LVL2_ECW_WEIGHTS['Critical Weight ILO'] = (
    ISCO_LVL2_ECW_WEIGHTS.index.astype(int)
    .isin(ILO_LVL2_ECW_GROUPS)
    .astype(int)
)


In [113]:
#ISCO_LVL2_ECW_WEIGHTS

In [114]:
farms = ISCO_LVL2_ECW_WEIGHTS.loc['62'] # Extrapolate all farming related roles from the single farming,
ISCO_LVL2_ECW_WEIGHTS.loc['61'] = farms # as the Indoor / Outdoor context is missing this data point
ISCO_LVL2_ECW_WEIGHTS.loc['63'] = 0 # Equate Subsitance farmers to 100% outdoor (Assumed as its not an industrial process)

In [115]:
# Convert the Series directly to a dictionary
ISCO_LVL2_ECW_WEIGHTS["ISCO_08_PollWeights"] = ISCO_LVL2_ECW_WEIGHTS['Critical Weight POLL']*ISCO_LVL2_ECW_WEIGHTS['Context Proj']
ISCO_LVL2_ECW_WEIGHTS["ISCO_08_ILOWeights"] = ISCO_LVL2_ECW_WEIGHTS['Critical Weight ILO']*ISCO_LVL2_ECW_WEIGHTS['Context Proj']

In [116]:
#ISCO_LVL2_ECW_WEIGHTS

#### 2. The following section extracts the ILO data source and complies a dictionary of countries and the given number of workers employed under each level 2 ISCO-08 code.

In [117]:
ILO_df = pd.read_csv(ilo_employment_by_isco_08_l2_PATH)

In [118]:
ILO_df['ISCO-8 L2 Code'] = ILO_df['classif1.label'].str.split(':').str[1].str[1:4]
ILO_df = ILO_df.rename(columns={'ref_area.label': 'Country', 'obs_value': 'Employment (thou)'})
ILO_df = ILO_df[['Country', 'ISCO-8 L2 Code', 'Employment (thou)', 'time']]
ILO_df['Employment'] = ILO_df['Employment (thou)'] * 1000
ILO_df = ILO_df.drop(columns=['Employment (thou)'])


In [119]:
#ILO_df

In [120]:
EmploymentByISOC8 = {}
for _, row in ILO_df.iterrows():
    c    = row['Country']
    code = row['ISCO-8 L2 Code']
    yr   = row['time']
    emp  = row['Employment']

    EmploymentByISOC8.setdefault(c, {}).setdefault(code, {})[yr] = emp

In [121]:
for code_dict in EmploymentByISOC8.values():
    for isco_code, year_dict in list(code_dict.items()):
        if isinstance(year_dict, dict) and year_dict:
            # Filter out NaN values before finding the latest year
            valid_years = {year: value for year, value in year_dict.items() if pd.notna(value)}
            if valid_years:
                latest = max(valid_years)
                code_dict[isco_code] = valid_years[latest]
            else:
                # Handle cases where all values for a given ISCO code are NaN
                code_dict[isco_code] = None # Or some other appropriate value

In [122]:
# Convert the Series directly to a dictionary
ISCO_08_PollWeights = ISCO_LVL2_ECW_WEIGHTS['ISCO_08_PollWeights'].to_dict()
ISCO_08_ILOWeights = ISCO_LVL2_ECW_WEIGHTS['ISCO_08_ILOWeights'].to_dict()


In [123]:
#ISCO_08_PollWeights

In [124]:
ECW_dict_Poll = {}

# Iterate through the nested_dict
for country, employment_dict in EmploymentByISOC8.items():
    ECW_dict_Poll[country] = {} # Initialize dictionary for the country
    for isco_code, employment in employment_dict.items():
        # Ensure isco_code is a string and strip whitespace for lookup
        isco_code_str = str(isco_code).strip()

        # Check if the ISCO code exists in the weights dictionary and employment is not NaN
        # The isco_weights dictionary keys are now strings (fixed in cell 4467391f)
        if isco_code_str in ISCO_08_PollWeights.keys() and pd.notna(employment):
            weight = ISCO_08_PollWeights[isco_code_str]
            # Check if both weight and employment are numeric before multiplication
            if pd.notna(weight):
                critical_employment = employment * weight
            else: critical_employment = 0
            ECW_dict_Poll[country][isco_code_str] = critical_employment # Use stripped code as key
        pass # print(f"Skipping {isco_code} in {country}: Weight missing or Employment is NaN")

In [125]:
# Calculate critical employment by multiplying employment by the weights
ECW_dict_ILO = {}

# Iterate through the nested_dict (which has structure {'Country': {ISCO Code: Employment}})
for country, employment_dict in EmploymentByISOC8.items():
    ECW_dict_ILO[country] = {} # Initialize dictionary for the country
    for isco_code, employment in employment_dict.items():
        # Ensure isco_code is a string and strip whitespace for lookup
        isco_code_str = str(isco_code).strip()

        # Check if the ISCO code exists in the weights dictionary and employment is not NaN
        # The isco_weights dictionary keys are now strings (fixed in cell 4467391f)
        if isco_code_str in ISCO_08_ILOWeights.keys() and pd.notna(employment):
            weight = ISCO_08_ILOWeights[isco_code_str]
            # Check if both weight and employment are numeric before multiplication
            if pd.notna(weight):
                critical_employment = employment * weight
            else: critical_employment = 0
            ECW_dict_ILO[country][isco_code_str] = critical_employment # Use stripped code as key
        pass # print(f"Skipping {isco_code} in {country}: Weight missing or Employment is NaN")

In [126]:
#ECW_dict_ILO['Australia']
#ECW_dict_Poll

In [127]:
ECW_ILO = {}

for country, isco_data in ECW_dict_ILO.items():
    ECW_ILO[country] = sum(isco_data.values())

In [128]:
ECW_Poll = {}

for country, isco_data in ECW_dict_Poll.items():
    ECW_Poll[country] = sum(isco_data.values())

In [129]:
ECW_Poll_pc = {}

for country, isco_data in EmploymentByISOC8.items():
    total_employment = isco_data['Tot']
    ECW_Poll_pc[country] = float(ECW_Poll[country]/total_employment)

In [130]:
ECW_ILO_pc = {}

for country, isco_data in EmploymentByISOC8.items():
    total_employment = isco_data['Tot']
    ECW_ILO_pc[country] = float(ECW_ILO[country]/total_employment)

In [131]:
#EmploymentByISOC8

# 3 Labour Force Data


##### 3.1 Import the most recent figures for labour force by ISO-3 countries. Missing countrys labour forces were manually aquired and there soruces can be found the in the "LFData_WB_plus.xlsx" sheet in the data folder

In [132]:
labourForce_df = pd.read_excel(world_bank_labour_force_by_country_PATH, usecols=[0, 1, 4])

In [133]:
labourForce_df['Region'] = cc.convert(labourForce_df['Country Name'].fillna('Nan').astype(str).tolist(), to='UNregion', not_found='not found')
labourForce_df.dropna(subset=['Country Name'], inplace=True) # append a column of the countrys UNregion.

Channel Islands not found in regex


In [134]:
labourForce_df.loc[labourForce_df["Country Name"] == "Channel Islands", "Region"] = "Northern Europe" # Manually append Channel Islands

In [135]:
# labourForce_df 
# the full data frame of ECW's by Country from both the Upper
# bound (strickly critcal works, aquired form the poll) and the Lower bound
# (Covid-style essential worker estimate, ILO-the value of essential work)

In [136]:
for idx, row in labourForce_df.iterrows():
    country = row["Country Name"]
    if country in ECW_ILO_pc.keys():
      labourForce_df.at[idx, "%ECW ILO"] = ECW_ILO_pc[country]
      labourForce_df.at[idx, "%ECW Poll"] = ECW_Poll_pc[country]

In [137]:
for i in ECW_Poll_pc.keys():
  if i not in labourForce_df['Country Name'].tolist():
    print(i)

Cook Islands
Montserrat
Niue
Tokelau
Wallis and Futuna


In [138]:
missing_code_list = []
missing_country_list = []
for idx, row in labourForce_df.iterrows():
    country = row["Country Name"]
    code = row["Country Code"]
    if pd.isna(row['%ECW ILO']):
      missing_code_list.append(code)
      missing_country_list.append(country)
    if pd.isna(row['%ECW Poll']):
      missing_code_list.append(code)
      missing_country_list.append(country)

###### The following dictionary is used to project missing labour force breakdown data (ISCO-08 breakdown), by taking the average of neighbouring countries with similar workfoces and GDP per Capita. The dictionary may need to be run multiple times if there are interdepenacies within the dictionary

In [139]:
similar_iso3 = {
    "ABW": ["CUW","SXM","MHL"],
    "AIA": ["VGB","TCA","MAF"],
    "AND": ["LIE","CYP","MCO"],
    "ARM": ["GEO","AZE","ALB"],
    "ASM": ["GUM","MNP","WSM"],
    "ATA": ["ATF","HMD","SGS"],
    "ATF": ["HMD","BVT","SGS"],
    "ATG": ["KNA","MDG","VCT"],
    "AZE": ["GEO","KAZ","UZB"],
    "BES": ["ABW","CUW","SXM"],
    "BHR": ["ARE","QAT","OMN"],
    "BLM": ["MAF","SXM","GLP"],
    "BMU": ["MHL"],
    "BVT": ["ATF","HMD","SGS"],
    "CAF": ["TCD","SSD","NER"],
    "CAN": ["USA"],
    "CCK": ["CXR","NFK","HMD"],
    "CHI": ["GBR"],
    "CHN": ["JPN","IND","VNM"],
    "CMR": ["COG","GAB","NGA"],
    "COG": ["GAB","CMR","GNQ"],
    "COM": ["MDG","MUS","SYC"],
    "CPV": ["STP","COM","MUS"],
    "CUB": ["JAM","DOM","PRI"],
    "CUW": ["ABW","MHL"],
    "CXR": ["CCK","NFK","HMD"],
    "CYM": ["VGB","TCA","BMU"],
    "DJI": ["ERI","SOM","YEM"],
    "DMA": ["KNA","VCT","LCA"],
    "DZA": ["MAR","TUN","LBY"],
    "ERI": ["DJI","SOM","SDN"],
    "ESH": ["MAR","MRT","DZA"],
    "FLK": ["SGS","SHN","BVT"],
    "FRO": ["ISL","GRL"],
    "FSM": ["MHL","KIR","PLW"],
    "GAB": ["GNQ","COG","AGO"],
    "GGY": ["JEY","IMN","BMU"],
    "GIB": ["MLT","AND","LIE"],
    "GLP": ["MTQ","MAF","BLM"],
    "GNQ": ["GAB","COG","STP"],
    "GRL": ["ISL","FRO"],
    "GUF": ["SUR","GUY","MTQ"],
    "GUM": ["MNP","ASM","PLW"],
    "HKG": ["MAC","SGP","CHN"],
    "HMD": ["ATF","BVT","SGS"],
    "HTI": ["NIC","JAM","HND"],
    "IMN": ["CHI", "GBR"],
    "IOT": ["HMD","CCK","CXR"],
    "JAM": ["BRB","TTO","BHS"],
    "JEY": ["GGY","IMN","BMU"],
    "KAZ": ["UZB","TKM","AZE"],
    "KOR": ["JPN","CHN"],
    "KNA": ["ATG","DMA","VCT"],
    "KWT": ["QAT","BHR","OMN"],
    "LBY": ["DZA","TUN","EGY"],
    "LCA": ["VCT","DMA","ATG"],
    "LIE": ["CYP","SMR","MCO"],
    "MAC": ["HKG","SGP","CHN"],
    "MAF": ["SXM","MDG"],
    "MAR": ["TUN","DZA","EGY"],
    "MCO": ["CYP","LIE","SMR"],
    "MDA": ["UKR","GEO","ALB"],
    "MLT": ["CYP","MNE","ISL"],
    "MNP": ["GUM","ASM","PLW"],
    "MRT": ["TCD","NER"],
    "MTQ": ["GLP","MAF","BLM"],
    "MWI": ["MOZ","ZMB","TZA"],
    "MYS": ["THA","IDN","VNM"],
    "MYT": ["REU","COM","MUS"],
    "NCL": ["WSM"],
    "NFK": ["CCK","CXR","HMD"],
    "NIC": ["HND","GTM","SLV"],
    "NZL": ["AUS"],
    "OMN": ["QAT","ARE","BHR"],
    "PCN": ["TKL","NIU","NFK"],
    "PRI": ["PAN","TTO","JAM"],
    "PRK": ["VNM","LAO","MMR"],
    "PRY": ["BOL","PER","URY"],
    "PYF": ["WSM"],
    "QAT": ["KWT","BHR","ARE"],
    "REU": ["MYT","MUS","COM"],
    "SAU": ["ARE","QAT","ARE"],
    "SGS": ["FLK","BVT","ATF"],
    "SHN": ["FLK","PCN","NFK"],
    "SJM": ["GRL","FRO","ISL"],
    "SLB": ["VUT","PNG","FJI"],
    "SMR": ["CYP","LIE","MCO"],
    "SPM": ["BMU","JEY","GGY"],
    "SSD": ["TCD","CAF","ERI"],
    "SXM": ["ABW","CUW","MAF"],
    "SYR": ["IRQ","JOR","LBN"],
    "TCA": ["CYM","VGB","ABW"],
    "TCD": ["CAF","NER","SSD"],
    "TKM": ["UZB","KAZ","AZE"],
    "TWN": ["KOR","JPN","HKG"],
    "UMI": ["PCN","NFK","CXR"],
    "UZB": ["KAZ","TKM","KGZ"],
    "VAT": ["SMR","MCO","CYP"],
    "VCT": ["LCA","DMA","ATG"],
    "VEN": ["COL","ECU","PER"],
    "VGB": ["CYM","TCA","ABW"],
    "VIR": ["VGB","ABW","CUW"],
    "YEM": ["SOM","SDN","ERI"],

}

In [140]:
cont = True
while cont:
    cont = False
    for idx, row in labourForce_df.iterrows():
        if pd.isna(row['%ECW ILO']):
            try:
                code = row["Country Code"]
                ave = []
                for iso in similar_iso3.get(code, []):  # safer: avoid KeyError
                    match = labourForce_df.loc[labourForce_df["Country Code"] == iso, '%ECW ILO']
                    if not match.empty:
                        z = match.iloc[0]   # safely take the first value
                        if not pd.isna(z):
                            ave.append(float(z))
                if ave:  
                    # update if we found at least one valid value
                    labourForce_df.at[idx, '%ECW ILO'] = sum(ave) / len(ave)
                else:
                    # couldn't fill this row → rerun
                    cont = True
            except KeyError:
                cont = True
                pass


In [141]:
cont = True
while cont:
    cont = False
    for idx, row in labourForce_df.iterrows():
        if pd.isna(row['%ECW Poll']):
            try:
                code = row["Country Code"]
                ave = []
                for iso in similar_iso3.get(code, []):  # safer: avoid KeyError
                    match = labourForce_df.loc[labourForce_df["Country Code"] == iso, '%ECW Poll']
                    if not match.empty:
                        z = match.iloc[0]   # safely take the first value
                        if not pd.isna(z):
                            ave.append(float(z))
                if ave:  
                    # update if we found at least one valid value
                    labourForce_df.at[idx, '%ECW Poll'] = sum(ave) / len(ave)
                else:
                    # couldn't fill this row → rerun
                    cont = True
            except KeyError:
                cont = True
                pass


In [142]:
labourForce_df['ECW ILO'] = (labourForce_df['%ECW ILO'] * labourForce_df[2024])
labourForce_df['ECW Poll'] = (labourForce_df['%ECW Poll'] * labourForce_df[2024])

In [143]:
print(f"{labourForce_df['ECW ILO'].sum(skipna=True):.2e}")

1.28e+09


In [144]:
labourForce_df

,Country Name,Country Code,2024,Region,%ECW ILO,%ECW Poll,ECW ILO,ECW Poll
0,Aruba,ABW,5.582600e+04,Caribbean,0.343921,0.139363,1.919971e+04,7.780081e+03
1,Afghanistan,AFG,9.130000e+06,Southern Asia,0.307968,0.123933,2.811747e+06,1.131504e+06
2,Angola,AGO,1.600000e+07,Middle Africa,0.275629,0.074852,4.410068e+06,1.197635e+06
3,Albania,ALB,1.370000e+06,Southern Europe,0.338142,0.151029,4.632548e+05,2.069104e+05
4,Andorra,AND,5.050400e+04,Southern Europe,0.315930,0.113649,1.595575e+04,5.739737e+03
5,United Arab Emirates,ARE,7.090000e+06,Western Asia,0.304209,0.119344,2.156844e+06,8.461496e+05
6,Argentina,ARG,2.230000e+07,South America,0.366796,0.112854,8.179552e+06,2.516635e+06
7,Armenia,ARM,1.510000e+06,Western Asia,0.297254,0.138317,4.488540e+05,2.088583e+05
8,American Samoa,ASM,5.595800e+04,Polynesia,0.290419,0.120597,1.625127e+04,6.748353e+03
9,Antigua and Barbuda,ATG,4.654000e+04,Caribbean,0.247786,0.151973,1.153198e+04,7.072801e+03


In [145]:
pd.set_option('display.max_rows', None)


# Final Result
###### This table is the relevant information we will be using as the total number of ECWs in a given region. This contains both a lower and an upper bound. The lower bound is sourced by an internal census between our team and the project hosts, ALLFED, to evaluate the cirticallity of ISCO-08 Level 4 codes. The Upper bound is similarly derived however the essential ISCO-08 Level 2 job codes were pulled from an ILO study on the effects of COVID-19.

In [146]:
regional_labourForce_df = labourForce_df.groupby('Region')[['ECW ILO', 'ECW Poll']].sum().reset_index()
display(regional_labourForce_df)

,Region,ECW ILO,ECW Poll
0,Australia and New Zealand,5.272682e+06,2.366326e+06
1,Caribbean,6.734296e+06,2.247684e+06
2,Central America,3.341591e+07,1.092332e+07
3,Central Asia,9.548740e+06,4.395989e+06
4,Eastern Africa,5.430956e+07,2.573874e+07
5,Eastern Asia,3.290157e+08,1.340928e+08
6,Eastern Europe,4.685251e+07,2.304591e+07
7,Melanesia,1.612686e+06,6.193071e+05
8,Micronesia,5.864985e+04,2.246516e+04
9,Middle Africa,1.964237e+07,9.369414e+06


In [151]:
results_folder = cwd.parent / "results"
output_file_region = results_folder / "ECWbyRegion.csv"
regional_labourForce_df.to_csv(output_file_region, index=False)

In [152]:
output_file_country = results_folder / "ECWbyCountry.csv"
labourForce_df.to_csv(output_file_country, index=False)

'South-eastern Asia'

In [160]:
countries = [
    "Albania", "Argentina", "Australia", "Austria", "Bangladesh", "Belarus",
    "Belgium", "Bosnia and Herzegovina", "Botswana", "Brazil", "Brunei", "Bulgaria",
    "Cambodia", "Canada", "Chile", "China", "Colombia", "Croatia", "Czech Republic",
    "Côte d'Ivoire", "DR Congo", "Denmark", "Djibouti", "Dominican Republic", "Egypt",
    "El Salvador", "Eswatini", "Ethiopia", "Finland", "France", "Georgia", "Germany",
    "Ghana", "Greece", "Guadeloupe", "Guatemala", "Guinea", "Honduras", "Hong Kong",
    "Hungary", "India", "Indonesia", "Iran", "Ireland", "Israel", "Italy", "Jamaica",
    "Japan", "Kazakhstan", "Kenya", "Kosovo", "Kyrgyzstan", "Laos", "Latvia", "Madagascar",
    "Malawi", "Malaysia", "Mauritius", "Mexico", "Moldova", "Mongolia", "Montenegro", "Morocco",
    "Mozambique", "Myanmar", "Namibia", "Netherlands", "New Caledonia", "New Zealand", "Niger",
    "Nigeria", "North Korea", "North Macedonia", "Oman", "Pakistan", "Panama", "Papua New Guinea",
    "Peru", "Philippines", "Poland", "Portugal", "Romania", "Russia", "Senegal", "Serbia",
    "Slovakia", "Slovenia", "South Africa", "South Korea", "Spain", "Sri Lanka", "Sudan", "Sweden",
    "Syria", "Taiwan", "Tajikistan", "Tanzania", "Thailand", "Türkiye", "Ukraine", "United Arab Emirates",
    "United Kingdom", "United States", "Uzbekistan", "Venezuela", "Vietnam", "Zambia", "Zimbabwe"
]


import country_converter as coco

# Initialize the converter
converter = coco.CountryConverter()

# Convert country names to UN regions
un_regions = converter.convert(names=countries, to='UNregion')



In [163]:
for i in un_regions: 
    print(i)

Southern Europe
South America
Australia and New Zealand
Western Europe
Southern Asia
Eastern Europe
Western Europe
Southern Europe
Southern Africa
South America
South-eastern Asia
Eastern Europe
South-eastern Asia
Northern America
South America
Eastern Asia
South America
Southern Europe
Eastern Europe
Western Africa
Middle Africa
Northern Europe
Eastern Africa
Caribbean
Northern Africa
Central America
Southern Africa
Eastern Africa
Northern Europe
Western Europe
Western Asia
Western Europe
Western Africa
Southern Europe
Caribbean
Central America
Western Africa
Central America
Eastern Asia
Eastern Europe
Southern Asia
South-eastern Asia
Southern Asia
Northern Europe
Western Asia
Southern Europe
Caribbean
Eastern Asia
Central Asia
Eastern Africa
Southern Europe
Central Asia
South-eastern Asia
Northern Europe
Eastern Africa
Eastern Africa
South-eastern Asia
Eastern Africa
Central America
Eastern Europe
Eastern Asia
Southern Europe
Northern Africa
Eastern Africa
South-eastern Asia
Southern